In [2]:
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np



In [3]:
start="2005-01-01"
end="2025-01-01"
stock="GOOG"

data=yf.download(stock,start,end)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [6]:
data.reset_index(inplace=True)

In [8]:
data

Price,Date,Close,High,Low,Open,Volume
Ticker,,GOOG,GOOG,GOOG,GOOG,GOOG
0,2005-01-03,5.025025,5.048079,4.845304,4.893394,636143518
1,2005-01-04,4.821506,5.030479,4.796220,4.992551,552298420
2,2005-01-05,4.796964,4.880999,4.765234,4.795476,330698912
3,2005-01-06,4.674010,4.856211,4.653434,4.835883,417041336
4,2005-01-07,4.805393,4.815309,4.679711,4.725819,387964757
...,...,...,...,...,...,...
5028,2024-12-24,197.345184,197.445061,194.975874,195.946768,6809800
5029,2024-12-26,196.875717,197.934509,195.647106,196.516126,7907900
5030,2024-12-27,193.819183,196.576052,191.753543,196.246426,14693000


In [10]:
data.dropna(inplace=True)


In [12]:
data_train = pd.DataFrame(data.Close[0 : int(len(data) * 0.85)])
data_test  = pd.DataFrame(data.Close[int(len(data) * 0.85) : len(data)])


In [14]:
data_train.shape[0]

4278

In [16]:
data_test.shape[0]

755

In [18]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))

In [19]:
data_train_scale=scaler.fit_transform(data_train)

In [22]:
x=[]
y=[]

for i in range(110,data_train_scale.shape[0]):
    x.append(data_train_scale[i-110:i])
    y.append(data_train_scale[i,0])

In [24]:
x,y=np.array(x),np.array(y)

In [26]:
from keras.layers import Dense,Dropout,LSTM
from keras.models import Sequential

In [27]:
model=Sequential()
model.add(LSTM(units=64,activation='relu',return_sequences=True,
               input_shape=(x.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=128,activation='relu',return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(units=256,activation='relu',return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(units=512,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(units=1))


C:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [28]:
model.compile(optimizer='adam',loss='mean_squared_error')

In [29]:
model.fit(x,y,epochs=100,batch_size=16,verbose=1)

Epoch 1/100
261/261 ━━━━━━━━━━━━━━━━━━━━ 91s 325ms/step - loss: 0.0093
Epoch 2/100
261/261 ━━━━━━━━━━━━━━━━━━━━ 84s 322ms/step - loss: 0.0016
Epoch 3/100
261/261 ━━━━━━━━━━━━━━━━━━━━ 55s 212ms/step - loss: 0.0012
Epoch 4/100
261/261 ━━━━━━━━━━━━━━━━━━━━ 54s 207ms/step - loss: 0.0013
Epoch 5/100
261/261 ━━━━━━━━━━━━━━━━━━━━ 58s 221ms/step - loss: 0.0014
Epoch 6/100
261/261 ━━━━━━━━━━━━━━━━━━━━ 56s 215ms/step - loss: 0.0014
Epoch 7/100
261/261 ━━━━━━━━━━━━━━━━━━━━ 62s 238ms/step - loss: 9.6013e-04
Epoch 8/100
261/261 ━━━━━━━━━━━━━━━━━━━━ 56s 213ms/step - loss: 0.0012
Epoch 9/100
261/261 ━━━━━━━━━━━━━━━━━━━━ 53s 205ms/step - loss: 0.0011
Epoch 10/100
261/261 ━━━━━━━━━━━━━━━━━━━━ 60s 229ms/step - loss: 8.2279e-04
Epoch 11/100
261/261 ━━━━━━━━━━━━━━━━━━━━ 63s 243ms/step - loss: 9.2256e-04
Epoch 12/100
261/261 ━━━━━━━━━━━━━━━━━━━━ 63s 240ms/step - loss: 9.2009e-04
Epoch 13/100
261/261 ━━━━━━━━━━━━━━━━━━━━ 63s 241ms/step - loss: 0.0010
Epoch 14/100
261/261 ━━━━━━━━━━━━━━━━━━━━ 61s 233ms/step 

KeyboardInterrupt: 

In [ ]:
pas_100_days=data_train.tail(100)

In [ ]:
data_test=pd.concat([pas_100_days,data_test],ignore_index=True)

In [ ]:
data_test_scale=scaler.fit_transform(data_test)

In [ ]:
x=[]
y=[]

for i in range(110,data_test_scale.shape[0]):
    x.append(data_test_scale[i-110:i])
    y.append(data_test_scale[i,0])

x,y=np.array(x),np.array(y)

In [ ]:
y_predict=model.predict(x)

In [ ]:
scale=1/scaler.scale_

In [ ]:
y_predict=y_predict*scale

In [ ]:
y=y*scale

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(y_predict,'r',label='PREDICTED PRICE')
plt.plot(y,'g',label='ORIGNAL PRICE')
plt.xlabel("time")
plt.ylabel("price")
plt.legend()
plt.show()